In [145]:
import pandas as pd
from pathlib import Path

In [146]:
spy_df = pd.read_csv(Path('HistoricalPrices.csv'), index_col='Date', parse_dates=True, infer_datetime_format=True)

In [147]:
spy_df.columns

Index([' Open', ' High', ' Low', ' Close', ' Volume'], dtype='object')

In [148]:
spy_df = spy_df.drop(columns=[' Open',' High',' Low',' Volume'])
spy_df = spy_df.rename(columns = {' Close':'Close'})

In [149]:
spy_df.index

DatetimeIndex(['2023-06-28', '2023-06-27', '2023-06-26', '2023-06-23',
               '2023-06-22', '2023-06-21', '2023-06-20', '2023-06-16',
               '2023-06-15', '2023-06-14',
               ...
               '2000-04-13', '2000-04-12', '2000-04-11', '2000-04-10',
               '2000-04-07', '2000-04-06', '2000-04-05', '2000-04-04',
               '2000-04-03', '2000-03-31'],
              dtype='datetime64[ns]', name='Date', length=5847, freq=None)

In [150]:
spy_df = spy_df.groupby(by=[spy_df.index.year, spy_df.index.month]).mean('Close')

In [151]:
#list comprehension to reformat multi-index 
lst_spy = [(f'{k}/{j}') for j,k in spy_df.index ]
spy_df.index = lst_spy

spy_df.index = pd.to_datetime(spy_df.index)
spy_df = spy_df.reset_index()
spy_df['index'] = spy_df['index'].apply(lambda x: x.strftime('%Y-%m'))
spy_df.set_index('index', inplace=True)
spy_df


,Close
index,
2000-03,150.375000
2000-04,146.231911
2000-05,142.080250
2000-06,146.521318
2000-07,147.503905
...,...
2023-02,407.072632
2023-03,396.023913
2023-04,410.921053


https://www.macrotrends.net/2637/lumber-prices-historical-chart-data

In [152]:
lumber_df = pd.read_csv(Path('lumber-prices-historical-chart-data.csv'), index_col='date', parse_dates=True, infer_datetime_format=True)

In [153]:
#calculate mean value for each month
lumber_df = lumber_df.groupby(by=[lumber_df.index.year, lumber_df.index.month]).mean()

In [154]:
#list comprehension to reformat multi-index 
lumber_lst = [(f'{k}/{j}') for j,k in lumber_df.index ]
lumber_df.index = lumber_lst

#make index datetime, reset the index so I can remove days, and put back into index
lumber_df.index = pd.to_datetime(lumber_df.index) 
lumber_df = lumber_df.reset_index()
lumber_df['index'] = lumber_df['index'].apply(lambda x: x.strftime('%Y-%m'))
lumber_df.set_index('index', inplace=True)
lumber_df.dropna(inplace=True)

In [155]:
lumber_df

,value
index,
1996-01,290.540909
1996-02,301.022000
1996-03,302.995238
1996-04,317.145714
1996-05,357.909091
...,...
2023-02,422.865000
2023-03,387.356522
2023-04,384.540000
